In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import random   

np.random.seed(42)
random.seed(42)

desire = 'Eat'
test_data_path = f'data/test/{desire}.csv'
test_df = pd.read_csv(test_data_path, index_col=False)
test_df.head(2)

,person_id,travel_purpose,target_amenity,mode,distance_miles,duration_minutes,age,individual_income,household_size,family_structure,vehicles,cypher
0,5.570035e+18,Eat,F&B Eatery/Drinking Places,Walking,0.0,0.0,29,53672,3_person,family_single,1,\n MATCH (p:Person)\n WHERE (p.a...
1,7.600486e+18,Eat,F&B Eatery/Snack and Nonalcoholic Beverage Bars,Walking,0.0,0.0,26,0,3_person,nonfamily_single,0,\n MATCH (p:Person)\n WHERE (p.a...


In [ ]:
from model import ChoiceModel
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd

num_sample = 900
num_threads = 3

data = []

choice_model = ChoiceModel(
    data_dir='data', desire=desire, sample_num=num_sample, skip_init=True)

def process_row(row):
    profile = row[['person_id', 'age', 'individual_income',
                    'household_size', 'family_structure', 'vehicles', 'cypher']]
    profile = profile.to_dict()
    response = choice_model.infer(
        profile=profile, mode='experiment',in_parallel=True)
    print(response)
    return response

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(process_row, row) for idx, row in test_df.iterrows()]
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            response = future.result()
            if response is not None:
                print('get response', response)
            data.append(response)
        except Exception as e:
            print(f"Error get response: {e}")

  0%|          | 1/1000 [03:42<61:49:20, 222.78s/it]

[5.570034953816602e+18, 'A 29 year old person, living in a family_single family with 3_person members. The person has 1 vehicles and an annual income of 53672 dollars.', 50, 'Eat', 'Boston', "\n        MATCH (p:Person)\n        WHERE (p.age>=18 AND p.age<30) OR (p.income>=30000 AND p.income<60000)\n        WITH p ORDER BY rand() LIMIT 50\n        MATCH (p)-[r1:WANT_TO]-(d:Desire {desire:'Eat'})-[r2:GO_TO]-(i:Intention)\n        RETURN COLLECT(DISTINCT p) AS person, \n            COLLECT(DISTINCT d) AS desire, \n            COLLECT(DISTINCT i) AS intention,\n            COLLECT(DISTINCT r1) AS want_to,\n            COLLECT(DISTINCT r2) AS go_to\n        ", [{'choice': 'Full-Service Restaurants', 'score': 0.42}, {'choice': 'Limited-Service Restaurants', 'score': 0.21}, {'choice': 'Nonalcoholic Bars', 'score': 0.19}, {'choice': 'Drinking Places', 'score': 0.05}], '{\n  "thought": "Based on the profile, this person seems to be a young adult living in Boston with a moderate income and famil

In [3]:
for idx, row in test_df.iterrows():
    process_row(row)
    break

[5.570034953816602e+18, 'A 29 year old person, living in a family_single family with 3_person members. The person has 1 vehicles and an annual income of 53672 dollars.', 50, 'Eat', 'Boston', "\n        MATCH (p:Person)\n        WHERE (p.age>=18 AND p.age<30) OR (p.income>=30000 AND p.income<60000)\n        WITH p ORDER BY rand() LIMIT 50\n        MATCH (p)-[r1:WANT_TO]-(d:Desire {desire:'Eat'})-[r2:GO_TO]-(i:Intention)\n        RETURN COLLECT(DISTINCT p) AS person, \n            COLLECT(DISTINCT d) AS desire, \n            COLLECT(DISTINCT i) AS intention,\n            COLLECT(DISTINCT r1) AS want_to,\n            COLLECT(DISTINCT r2) AS go_to\n        ", [{'choice': 'Full-Service Restaurants', 'score': 0.48}, {'choice': 'Nonalcoholic Bars', 'score': 0.19}, {'choice': 'Limited-Service Restaurants', 'score': 0.16}, {'choice': 'Drinking Places', 'score': 0.05}], '{ \n    "thought": "Based on the profile, a person who is A 29 year old person living in a family_single family with 3_person 

In [ ]:
data_df = pd.DataFrame(data, columns=['person_id', 'profile', 'top_k', 'desire', 'city', 'cypher',
                                      'amenity_recommendation', 'amenity_llm_choice', 'amenity_final_choice',
                                      'mode_recommendation', 'mode_llm_choice', 'mode_final_choice'])